## 目的地訪問率分析 ― 訪日外国人の空間的行動構造の変化を捉える ―

### 目的とアジェンダ

#### 分析の目的
訪日外国人が「どの地域を訪問しているか」という目的地選択の観点から、インバウンド需要の空間的な集中・分散構造の変化を把握する。

特に、コロナ前後での訪問率の変化や順位変動を通じて、単なる回復ではなく「選ばれる地域構造の変化」が生じているかを検証する。

#### 主な分析ステップ
1. 年別訪問率ランキングと集中構造の確認
2. 2019年比による訪問率の増減分析
3. 順位変動による相対的ポジションの変化
4. 主要都道府県の時系列推移


### 環境構築とデータ準備

訪問率データ（年 × 都道府県）を用いて分析を行う。

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

# データ読込（notebooks/ 配下から data/ を参照）
df = pd.read_csv("../data/inbound_destination.csv")

# 列名を統一
df = df.rename(columns={
    "Year": "year",
    "Prefecture": "prefecture",
    "Visit Rate(%)": "visit_rate"
})

PRE_COVID_YEAR = 2019
LATEST_YEAR = int(df["year"].max())

print("対象年:", sorted(df["year"].unique()))
print("都道府県数:", df["prefecture"].nunique())
print("最新年:", LATEST_YEAR)

df.head()

対象年: [np.int64(2018), np.int64(2019), np.int64(2023), np.int64(2024)]
都道府県数: 47
最新年: 2024


,year,prefecture,visit_rate
0,2018,東京,40.8239
1,2018,大阪,40.2257
2,2018,千葉,32.8209
3,2018,京都,29.7160
4,2018,福岡,11.3981


### 1. 年別訪問率ランキングと集中構造

各年における訪問率上位都道府県を確認し、
訪問の集中構造がどの程度維持されているかを把握する。

In [4]:
# ジニ係数（不均等度）の計算：分布の不均等さ
def gini(x):
    x = np.array(x, dtype=float)
    x = np.sort(x)
    n = len(x)
    if x.sum() == 0:
        return 0.0
    cumx = np.cumsum(x)
    return (n + 1 - 2 * (cumx.sum() / cumx[-1])) / n

# HHI（ハーフィンダール・ハーシュマン指数）の計算：シェアの集中度
def hhi_share(x):
    x = np.array(x, dtype=float)
    if x.sum() == 0:
        return 0.0
    s = x / x.sum()
    return float(np.sum(s**2))

rows = []

for y in sorted(df["year"].unique()):
    d = df[df["year"] == y].sort_values("visit_rate", ascending=False)
    v = d["visit_rate"].values
    
    rows.append({
        "year": y,
        "TOP5合計": v[:5].sum(),
        "TOP10合計": v[:10].sum(),
        "HHI": hhi_share(v),
        "Gini": gini(v)
    })

df_concentration = pd.DataFrame(rows)
df_concentration

,year,TOP5合計,TOP10合計,HHI,Gini
0,2018,154.9846,197.4529,0.093991,0.724074
1,2019,165.1994,207.2698,0.094599,0.724321
2,2023,175.2541,212.7440,0.104225,0.737969
3,2024,171.4049,211.8813,0.103687,0.741568


TOP10合計は2019年と比較して最新年でも大きく低下しておらず、訪問地は依然として一部の主要都道府県に集中している。

HHI・Gini係数が最新年で上昇していることから、コロナ後の回復局面では「分散」よりも「王道ルート回帰」が先行している可能性が示唆される。

### 2. 2019年比による訪問率の増減分析

2019年と最新年を比較し、訪問率が伸びている都道府県と
相対的に低下している都道府県を抽出する。

In [8]:
base = df[df["year"] == PRE_COVID_YEAR].set_index("prefecture")["visit_rate"]
latest = df[df["year"] == LATEST_YEAR].set_index("prefecture")["visit_rate"]

df_diff = pd.DataFrame({
    "2019年": base,
    f"{LATEST_YEAR}年": latest
})

df_diff["前年差"] = df_diff[f"{LATEST_YEAR}年"] - df_diff["2019年"]
df_diff["増加率"] = df_diff[f"{LATEST_YEAR}年"] / df_diff["2019年"]

df_diff.sort_values("前年差", ascending=False).head(15)

,2019年,2024年,前年差,増加率
prefecture,,,,
東京,42.3668,48.3346,5.9678,1.140860
千葉,32.2894,35.5781,3.2887,1.101851
福岡,9.0134,11.8966,2.8832,1.319879
山梨,6.5466,8.5451,1.9985,1.305273
神奈川,6.5724,8.4933,1.9209,1.292268
広島,3.0268,3.9738,0.9470,1.312872
大分,3.9471,4.3939,0.4468,1.113197
埼玉,0.6406,0.8346,0.1940,1.302841
熊本,2.1567,2.3381,0.1814,1.084110


首都圏（東京・千葉・神奈川）や福岡などの大都市・ゲートウェイは、2019年比で訪問率が上振れており、単なる回復を超えた集積が確認できる。

一方で、北海道や沖縄などの遠隔地は2019年水準を下回っており、回復局面における訪問地選択の優先順位が変化している可能性がある。

### 3. 順位変動による相対ポジションの変化

訪問率の絶対値だけでなく、順位の変化を通じて「相対的に選ばれ始めた地域」を把握する。


In [9]:
def rank_series(df, year):
    s = df[df["year"] == year].set_index("prefecture")["visit_rate"]
    s = s.sort_values(ascending=False)
    r = pd.Series(range(1, len(s) + 1), index=s.index)
    return s, r

s19, r19 = rank_series(df, PRE_COVID_YEAR)
s24, r24 = rank_series(df, LATEST_YEAR)

df_rank = pd.DataFrame({
    "訪問率_2019": s19,
    "訪問率_最新年": s24,
    "順位_2019": r19,
    "順位_最新年": r24
})

df_rank["順位変動（＋上昇）"] = df_rank["順位_2019"] - df_rank["順位_最新年"]

df_rank.sort_values("順位変動（＋上昇）", ascending=False).head(15)

,訪問率_2019,訪問率_最新年,順位_2019,順位_最新年,順位変動（＋上昇）
prefecture,,,,,
山形,0.4181,0.5651,39,32,7
宮城,1.0160,1.1590,28,22,6
栃木,1.0999,1.1752,26,21,5
福島,0.2477,0.3705,44,39,5
愛媛,0.3676,0.4144,42,38,4
埼玉,0.6406,0.8346,31,27,4
山梨,6.5466,8.5451,11,7,4
群馬,0.3874,0.4509,40,37,3
福岡,9.0134,11.8966,8,5,3


山梨県や福岡県などは順位を上げており、絶対値だけでなく相対的にも訪問先としての存在感を高めている。
これらの地域は、自然資源や都市近接性を背景に行動構造の変化を受け止め始めている可能性がある。

### 4. 主要都道府県の訪問率時系列推移

代表的な都道府県を抽出し、訪問率の推移を時系列で確認する。

In [10]:
targets = ["東京", "大阪", "京都", "千葉", "福岡", "山梨", "北海道", "沖縄"]

df_ts = df[df["prefecture"].isin(targets)]

fig = px.line(
    df_ts,
    x="year",
    y="visit_rate",
    color="prefecture",
    markers=True,
    title="主要都道府県の訪問率推移",
    labels={
        "year": "年",
        "visit_rate": "訪問率（%）",
        "prefecture": "都道府県"
    }
)

fig.update_xaxes(dtick=1, tickformat="d")
fig.show()

### 5. 結論とインサイト

1. 訪日外国人の訪問地は依然として主要都市圏に強く集中しており、コロナ後の回復局面ではその傾向がむしろ強まっている。
2. 一方で、山梨や福岡など一部の地域では、2019年比・順位変動の両面で改善が見られ、行動構造の変化を反映した新たな選択先となりつつある可能性がある。